# ANOVA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.stats as sm_stats
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

from statsmodels.stats.multicomp import MultiComparison, pairwise_tukeyhsd

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#rozkład $F$-Snedecora
Zmienna losowa $X$ ma rozkład $F$-Snedecora z parametrami $m,n\in\mathbb{N_+}$, jeżeli jej gęstość $f$~jest postaci

\begin{equation*}
      f(x)=
    \begin{cases}
    \frac{\Gamma(\frac{n+m}{2})}{\Gamma(\frac{n}{2})\Gamma(\frac{m}{2})}\Big(\frac{m}{n}\Big)^{\frac{m}{2}}x^{\frac{n}{2}-1}\Big(x+\frac{m}{n}\Big)^{-\frac{n+m}{2}} \quad &\text{dla} \quad x\geq0\\
    0 \quad &\text{dla} \quad x< 0,
    \end{cases}
\end{equation*}

Wartość oczekiwana i wariancja dane są wzorami
\begin{equation*}
    \mathbb{E}X = \frac{m}{m-2}, \quad \text{Var}(X) = \frac{2m^2(n+m-2)}{n(m-2)(m-4)}.
\end{equation*}

# Zadanie wstępne

1. Zdefiniuj zmienną losową dla rozkładu $F$-Snedecora.
2. Narysuj gęstość dla kilku wybranych stopni swobody. Zwróć uwagę na dziedzinę gęstości.
3. Narysuj dystrybuantę dla kilku wybranych stopni swobody.
4. Wyznacz podstawowe charakterystyki rozkładu: średnia, wariancja, odchylenie standardowe.
5. Wylosuj $n$ obserwacji z zdefiniowanego rozkładu.
6. Narysujmy histogram dla wylosowanej powyżej próbki.
7. Sprawdź, czy zachodzi własność: jeśli $X$ pochodzi z rozkładu chi-kwadrat o $m$ stopniach swobody i $Y$ również pochodzi z rozkładu chi kwadrat ale o $n$ stopniach swobody, oraz zmienne $X$ i $Y$ są niezależne, to zmienna 
$$
    Z = \frac{\frac{X}{m}}{\frac{Y}{n}}
$$
ma rozkład $F$-Snedecora o $(m,n)$ stopniach swobody.
8. Niech $X_1, \ldots, X_{n_1}$ będzie próbą prostą z rozkładu $\mathcal{N}(\mu_1, \sigma)$ oraz niech $Y_1, \ldots, Y_{n_2}$ będzie próbą prostą z rozkładu $\mathcal{N}(\mu_2, \sigma)$. Załóżmy ponadto, że obie próbki są niezależne. Niech $S_1^2$ i $S_2^2$ oznaczają, odpowiednio, wariancje próbkowe z pierwszej i drugiej próbki. Pokazać, że wówczas iloraz wariancji próbkowych 
$$
\frac{S_1^2}{S_2^2}
$$
ma rozkład $F$-Snedecora o $(n_1-1, n_2-1)$ stopniach swobody.

# Zadanie 1
Wykonano po cztery niezależne pomiary wytrzymałości na ́sciskanie trzech rodzajów betonu. Otrzymano następujące wyniki (w kG/cm2):

|    |  | | |  |
|:---:|:---:|:---:|:---:|:---:|
|  I  | 204 | 200 | 198 | 204 |
|  II | 197 | 205 | 213 | 209 |
| III | 190 | 208 | 202 | 210 |

Stwierdź, czy badane gatunki betonu różnią się istotnie pod względem ́sredniej wytrzymałości na ́sciskanie. Przyjmij poziom istotności 0.05.

Zanim przystapimy do testu $F$ powinnismy sprawdzic, czy sa spełnione założenia pozwalajace posłuzyc sie wspomnianą metodą:

1.  niezależność obserwacji,
2.  normalność  rozkładów  w  każdej  z  podpopulacji  wyznaczonych  przez  poziomy czynników,
3.  jednorodność wariancji podpopulacji wyznaczonych przez poziomy czynników.

## Jednorodność wariancji

Testujemy hipotezę zerową
$$
    H_0: \sigma_1^2 = \sigma^2_2 = \ldots = \sigma_r^2
$$

Do weryfikacji tej hipotezy możemy użyć testów:

1. [Bartlett's Test](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.bartlett.html)
2. [Levene Test for Equality of Variances](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.levene.html)


In [ ]:
?stats.bartlett

In [ ]:
?stats.levene

### Zadanie 
Napisz funkcję, która dla $n$ wektorów wykona i wypisze wyniki testu Bartletta i Levene.

#Normalność rozkładów
Testujemy hipotezę zerową
$$
    H_0: X_i\sim N(\mu_i,\sigma_i)
$$
Dla każdej grupy $i$.
Do weryfikacji tej hipotezy możemy użyć testów:

1. [Shapiro-Wilk Test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)
2. [Jarque-Bera Test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.jarque_bera.html)
3. [Kolmogorov-Smirnov Test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html)

In [ ]:
?stats.shapiro

In [ ]:
?stats.jarque_bera

In [ ]:
?stats.kstest

# ANOVA

Testujemy hipotezę zerową
$$
H_0: \mu_1 = \mu_2 = \mu_3
$$ 
mówiącą, iż - ́sredniorzecz biorąc – badane gatunki betonu nie różnią się istotnie pod względem wytrzymałości na ́ściskanie, wobec hipotezy alternatywnej orzekającej, iż te gatunki różnią się wytrzymałością.

Oznaczmy (w naszym zadaniu $r=3,n=4$)





*   Średnią ze wszystkich obserwacji
    \begin{equation*}
        \overline{\overline{Y}} = \frac{1}{nr}\sum_{i=1}^r\sum_{j=1}^n Y_{ij},
    \end{equation*}
*   Średnią na $i$-tym poziomie, $1 \leq i \leq n$
    \begin{equation*}
        \overline{Y}_i = \frac{1}{n}\sum_{j=1}^n Y_{ij},
    \end{equation*}
*       Zmienność całkowitą w eksperymencie
    \begin{equation*}
        \text{SST} = \sum_{i=1}^r\sum_{j=1}^n (Y_{ij} -  \overline{\overline{Y}})^2,
    \end{equation*}
*   zmienność między poziomami
    \begin{equation*}
        \text{SSA} = n\sum_{i=1}^r(\overline{Y}_i - \overline{\overline{Y}})^2,
\end{equation*}
*  zmienność na danym poziomie
    \begin{equation*}
        \text{SSE} = \sum_{i=1}^r\sum_{j=1}(Y_{ij} - \overline{Y}_i)^2.
    \end{equation*}

    TW.  $\text{SST}=\text{SSA}+\text{SSE}$


Statystyka testowa jest postaci:
    \begin{equation*}
        F = \frac{\frac{\text{SSA}}{r-1}}{\frac{SSE}{r(n-1)}}=\frac{MSA}{MSE}.
    \end{equation*}

Obszar krytyczny testu $F$ jest postaci
\begin{equation*}
    [F^{(r-1)(r(n-1))}_{1-\alpha}, +\infty).
\end{equation*}

# *Zadanie 1b
Zaimplementuj swoją wersję testu $F$.

# Zadanie 2
Wapń jest niezbędnym minerałem, który reguluje pracę serca, jest ważny dla krzepnięcia krwi i budowy zdrowych kości. Narodowa Fundacja Osteoporozy zaleca dzienne spożycie wapnia w ilości 1000-1200 mg / dzień dla dorosłych mężczyzn i kobiet. Podczas gdy wapń jest zawarty w niektórych produktach spożywczych, większość dorosłych nie ma wystarczającej ilości wapnia w swojej diecie i przyjmuje suplementy. Niestety niektóre suplementy mają skutki uboczne, takie jak zaburzenia żołądkowe, co utrudnia ich regularne przyjmowanie przez niektórych pacjentów.

Badanie ma na celu sprawdzenie, czy istnieje różnica w średnim dziennym spożyciu wapnia u osób dorosłych z prawidłową gęstością kości, osób dorosłych z osteopenią (mała gęstość kości, która może prowadzić do osteoporozy) i osób dorosłych z osteoporozą. Dorośli w wieku 60 lat z prawidłową gęstością kości, osteopenią i osteoporozą są losowo wybierani z rejestrów szpitalnych i zapraszani do udziału w badaniu. Dzienne spożycie wapnia przez każdego uczestnika jest mierzone na podstawie zgłoszonego spożycia żywności i suplementów. Dane są pokazane poniżej.

| Prawidłowa gęstość kości | Osteopenia | Osteoporoza |
|:------------------------:|:----------:|:-----------:|
|           1200           |    1000    |     890     |
|           1000           |    1100    |     650     |
|            980           |     700    |     1100    |
|            900           |     800    |     900     |
|            750           |     500    |     400     |
|            800           |     700    |     350     |

Czy istnieje statystycznie istotna różnica w średnim spożyciu wapnia u pacjentów z prawidłową gęstością kości w porównaniu do pacjentów z osteopenią i osteoporozą? Przyjmij poziom istotności 0.05.

## post-hoc - porówania wielokrotne
Testujemy
\begin{equation*}
    \begin{split}
        H_{0, ik}&: \mu_i = \mu_k, \\
        H_{1, ik}&: \mu_i \neq \mu_k
    \end{split}
\end{equation*}
dla wszystkich par $i, k=1, \ldots, r, i < k$.

Hipotezę zerową odrzucamy, gdy $|\overline{Y}_i - \overline{Y}_k|$ jest ,,za duża''.

### Metoda Tukeya
Odrzucamy hipotezę zerową $H_{0, ik}$, gdy
\begin{equation*}
    |\overline{Y}_i - \overline{Y}_k| \geq q_{1-\alpha}^{(r, r(n-1))}\sqrt{\frac{\text{MSE}}{n}},
\end{equation*}
gdzie $q_{1-\alpha}^{(r, r(n-1)}$ jest kwantylem studentyzowanego rozkładu rozstępu.

# Zadanie 3
Wysunięto przypuszczenie, że wytrzymałość toreb papierowych zależy od stężenia twardego drewna w ścierze drzewnym, z którego owe torby są wytwarzane. W praktyce stosuje się stężenia od 5% do 20%. Dotychczasowe badania wykazały, że rozkład wytrzymałości toreb jest normalny, przy czym obserwowana zmienność nie zależała od poziomu stężenia twardego drewna w ścierze drzewnym. Poniższa tabela zawiera wyniki pomiarów wytrzymałości na rozciąganie (w psi) otrzymane podczas badań laboratoryjnych 24 toreb papierowych. Zbadać, czy wytrzymałość toreb zależy faktycznie od poziomu stężenia twardego drewna w ścierze drzewnym.

|  Stężenie ścieru drzewnego |    |    |    |    |    |    |
|:--------------------------:|:--:|:--:|:--:|:--:|:--:|:--:|
|             5%             |  7 |  8 | 15 | 11 |  9 | 10 |
|             10%            | 12 | 17 | 13 | 18 | 19 | 15 |
|             15%            | 14 | 18 | 19 | 17 | 16 | 18 |
|             20%            | 19 | 25 | 22 | 23 | 18 | 20 |

# Zadanie 4
W pliku `zarobki.csv` zamieszczono historyczne dane dotyczące wysokosci miesięcznych zarobków wybranych losowo osób w czterech miastach: w Warszawie, Krakowie, Wrocławiu i Katowicach. Zbadaj, czy wysokość miesięcznych zarobków tych miastach różni się istotnie (przyjmij poziom istotności 0.05).

# Zadanie 5
W celu porównania trzech ́srodków antykorozyjnych pobrano po 10 próbek losowych drutu zabezpieczanego każdym z tych ́srodków i zmierzono głębokość zaistniałej korozji (razy 10−3mm). Wyniki pomiarów przestawia poniższa tabelka:

| Środek A | Środek B | Środek C |
|:--------:|:--------:|:--------:|
|   98.5   |   100.2  |   56.7   |
|   98.5   |   99.2   |   82.0   |
|   98.5   |   99.9   |   67.8   |
|   97.5   |   97.8   |   58.3   |
|   99.3   |   99.8   |   61.2   |
|   102.0  |   100.5  |   67.8   |
|   101.8  |   99.8   |   117.4  |
|   98.3   |   99.0   |   103.4  |
|   102.0  |   101.1  |   43.8   |
|   101.2  |   100.8  |   86.1   |

Czy na podstawie tych  danym można stwierdzić, ̇ze ́srodki te różnią się istotnie podwzględem jakości tworzonego przez nie zabezpieczenia antykorozyjnego?

#Dwuczynnikowa anova
Model:
\begin{equation*}
        Y_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},
\end{equation*}
Testujemy hipotezy
\begin{equation*}
    \begin{split}
        H_{0,1}&: \alpha_1 = \alpha_2 = \ldots = \alpha_r=0 \\
        H_{1, 1}&: \neg H_{0, 1}, \\
                & \\
        H_{0,2}&: \beta_1 = \beta_2 = \ldots = \beta_s=0 \\
        H_{1,2}&: \neg H_{0,2},\\
                & \\
        H_{0,3}&: \gamma_{11} =  \ldots = \gamma_{rs}=0 \\
        H_{1,3}&: \neg H_{0,3}
    \end{split}
\end{equation*}

| Źródło zmienności | Suma kwadratów ochyleń | Stopnie swobody| Średni kwadrat ochyleń | Statystyki testowe |
|:--------:|:--------:|:--------:|:--------:|:--------:|
| Czynnik A | $SSA$| r − 1| MSA = $\frac{SSA}{r−1}$ | F1 = $\frac{MSA}{MSE}$ |
| Czynnik B | $SSB$| r − 1| MSB = $\frac{SSB}{s−1}$ | F1 = $\frac{MSB}{MSE}$ |
| Interakcje SSAB | $SSAB$| (r − 1)(s − 1)| MSAB = $\frac{SSAB}{(r−1)(s−1)}$ | F1 = $\frac{MSAB}{MSE}$ |
| Błąd losowy | $SSE$| rs(n − 1)| MSE = $\frac{SSE}{rs(n − 1)}$ |  |
| Ogółem | $SST$| rsn − 1|  |  |

# Zadanie 6
Przeprowadzono następujące doświadczenie: 18 mężczyzn i 18 kobiet rozmieszczono losowo w 9 pokojach w ten sposób, ̇ze w każdym pokoju były po dwie osoby tej samej płci. W pokojach tych utrzymywano stałą temperaturę: 18, 21 albo 24 stopnie Celsjusza (przydział temperatur poszczególnym pokojom był także losowy). Po upływie trzech godzin oceniano samopoczucie każdej z badanych osób (zastosowano ocenę punktową, w której 1 = zbyt zimno, 8 = idealna temperatura, 15 = zbyt ciepło).

|    |         M         |          K         |
|:--:|:-----------------:|:------------------:|
| 18 |  5, 4, 5, 4, 4, 2 |  1, 2, 5, 5, 1, 3  |
| 21 |  8, 8, 6, 3, 5, 7 |  10, 7, 8, 8, 7, 8 |
| 24 | 12, 8, 8, 7, 6, 6 | 11, 13, 8, 8, 6, 7 |

Zbadaj wpływ, jaki na samopoczucie osób wywiera temperatura panująca w danym pokoju. Czy ocena samopoczucia zależy od płci? Czy występują ̨tu istotne interakcje między badanymi czynnikami (tzn. temperaturą i płcią)?

# Zadanie 7
W  pewnych  zakładach  lotniczych  stosuje  się  dwie  metody  nakładania farby podkładowej na częsci aluminiowe: malowanie zanurzeniowe i natryskowe. Czyni się  tak  w  celu  zwiększenia  przylegania  właściwej  farby  nawierzchniowej,  którą  później są malowane owe części. We wspomnianych zakładach stosowano do tej pory trzy rodzaje farb podkładowych. Inżynier technolog, odpowiedzialny za ten etap produkcji, postanowił zbadać, czy rodzaj farby podkładowej oraz sposób jej nakładania na detal mają istotny wpływ na siłę przylegania właściwej farby nawierzchniowej. W tym celu przeprowadzono eksperyment, w którym zmierzono siłę przylegania farby nawierzchniowej do kilku detali malowanych najpierw różnymi farbami podkładowymi, nanoszonymi obiema metodami. Wyniki pomiarów zamieszczono w poniższej tabeli.

| Rodzaj farby |  Malowanie zanurzeniowe |  Malowanie natryskowe |
|:------------:|:-----------------------:|:---------------------:|
|       A      |      4.0, 4.5, 4.3      |     5.4, 4.9, 5.6     |
|       B      |      5.6, 4.9, 5.4      |     5.8, 6.1, 6.3     |
|       C      |      3.8, 3.7, 3.9      |     6.5, 6.0, 5.0     |

Jakie wnioski powinien wyciągnąć inżynier na podstawie powyższych wyników?

# Zadanie 8
W celu zbadania wpływu czterech dawek nawożenia azotowego (w ilościach 0, 40, 80 i 120 kg/ha) na wysokość plonów lucerny przy trzech sposobach siewu (siew czysty C oraz dwa rodzaje wsiewu M i P w jęczmień jary) założono doświadczenie w czterech powtórzeniach. Dla każdej kombinacji nawożenia ze sposobem siewu zmierzono plon zielonej masy (w kg z poletka). W pierwszym pokosie uzyskano następujące obserwacje:

|   |            0            |           40           |           80           |           120          |
|:-:|:-----------------------:|:----------------------:|:----------------------:|:----------------------:|
| C | 33.2, 36.2, 44.2, 51.0  | 42.2, 41.4, 50.6, 45.2 | 50.2, 53.0, 52.6, 45.0 | 46.2, 52.4, 49.0, 43.6 |
| M |  18.6, 13.0, 14.6, 18.8 | 18.0, 20.0, 14.2, 19.1 | 24.2, 21.6, 16.4, 19.0 | 34.2, 17.2, 15.5, 22.2 |
| P |  20.4, 14.4, 11.0, 22.6 | 21.9, 42.0, 16.2, 25.6 | 18.2, 21.0, 27.3, 27.6 | 16.4, 15.0, 21.6, 27.8 |

Ustal, który z badanych czynników miał istotny wpływ na wysokość plonów masy zielonej.